In [ ]:
! pip install dtreeviz

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt
import seaborn as sns

eljest = {'natt': '#0E171B',
          #'kotte': '#422A25',
          'kotte': '#4E322C',
          'barr': '#50933B',
          'skymning': '#3B7C93',
          'gryning': '#BD6D24'}

## Feature information


#### Target
- total_fuel - Total fuel consumed in litres during the trip. Calculated from the fuel rate data in the raw VED data.

### Features
- total_time - Total duration in seconds of the trip, calculated from the tracking timestamp

- total_distance - Total driven distance in km of the trip, calculated as the sum of the $\Delta$Haversine distances between lat, lon points in the tracking data

- avg_speed - Average speed in km/h during the trip, calculated as a mean of the wheel-based speed signal

- std_speed - Standard deviation of all wheel-based speed readings in kmph during the trip

- max_speed - Maximum wheel-based speed reading during the trip

- min_acc - Minimum acceleration during the trip in m/s$^{-2}$

- max_acc - Maximum acceleration during the trip in m/s$^{-2}$

- std_acc - Standard deviation of the acceleration signal during the trip in m/s$^{-2}$, trying to capture jittery driving.

- n_harsh_accelerations - Number of times during the trip that the acceleration signal has exceeded 5 m/s$^{-2}$

- hour_of_day - The most common hour value among all the timestamps in the trip trajectory

- generalized_weight - Estimated mass of the vehicle, taken from the static raw VED data.

- engine_volume - The engine volume in litres of the vehicle, regexed from the engine type string in the static raw VED data.

Note: *The acceleration signal was calculated by numerical differentiation of the wheel-based speed w.r.t. time. The typical $\Delta t$ of the raw data is slightly less than 1 sec. Both the velocity signal and the resulting acceleration signal have been treated with a moving average of 3 periods to reduce the noise somewhat. But the acceleration signal is still very spiky and unrealstic and the whole calculation would ideally need to be filtered more to be realistic.*


In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/signalfel/xaiclinic/main/ved_ice_trips.csv')

In [ ]:
X = data.iloc[:, 1:]
y = data.total_fuel

In [ ]:
X.shape

In [ ]:
X.head()

In [ ]:
# Perform the train-test split. Everyone uses the same random_state so that we
# can easily compare results
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1337)

# **Part 1 - Linear Regression**

### Train a Linear Regression model

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

# Train the model on the training set (Note that we don't
# perform any feature scaling, such as removing the mean and forcing unit std)
lr.fit(X_train, y_train);

# Print the R2 score for the test set (should be 0.86)
print(r2_score(lr.predict(X_test), y_test).round(3))

### TODO❗ What do the weights in a linear model tell us?

Answer:

#### Let's calculate and plot the weights

In [ ]:
weights = lr.coef_
df_weights = pd.DataFrame({'weight': weights}, index=X_train.columns)

In [ ]:
df_weights.plot(kind='barh', color=eljest['kotte'])

### TODO❗ Create an effect plot from the weights

The effects of a feature on a single data instance $i$ is calculated as

\begin{equation}
E_j^{(i)} = w_j x_j^{(i)}
\end{equation}

where $w_j$ is the weight of feature $j$ and $x_j^{(i)}$ is the value of the feature $j$ for data instance $i$.

There will of course be as many effects as there are data instances, so we need to figure out how to visualize it.

In [ ]:
effects = ...

### TODO❗ What does the Effect plot tell us that the weights do not?

Answer:

# **Part 2 - Understanding tree models**



In [ ]:
from sklearn.tree import DecisionTreeRegressor

# We start with a very shallow tree to be able to visualize it
tree_depth = 3

dtree = DecisionTreeRegressor(criterion='squared_error',
                                    max_depth=tree_depth,
                                    random_state=1337)


# Again, no feature scaling
dtree.fit(X_train, y_train);

# Print the R2 score for this very shallow tree, depth = 3 (should be 0.694)
print(r2_score(dtree.predict(X_test), y_test).round(3))

### graphviz

In [ ]:
from sklearn import tree
import graphviz

g = tree.export_graphviz(dtree,
                         feature_names=X_train.columns.tolist(),
                         filled=True)
graphviz.Source(g)

### dtreeviz

In [ ]:
import dtreeviz

viz = dtreeviz.model(dtree, X_train, y_train,
                     target_name='total_fuel',
                     feature_names=X_train.columns)
viz.view(fontname='monospace')

### TODO❗ Which of the two do you prefer? What are some pros and cons of either?

Answer:

### TODO❗ Let's investigate how the performance varies when we make the tree deeper. Finish the for loop (sorry, not very engaging, but hey)

In [ ]:
r2_scores = []
tree_depths = list(range(2, 16))
random_state = 1337

for tree_depth in tree_depths:
  ...

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
plt.plot(tree_depths, r2_scores, color=eljest['barr'], marker='o')
plt.ylim([0.6, 0.9])
plt.xlim([tree_depths[0], tree_depths[-1]])
plt.xticks(tree_depths);

plt.legend(['R2 score'])
plt.xlabel('Tree depth');
plt.ylabel('Score');
plt.title('Model performance');

### TODO❗ How feasible is it to use the above methods to explain a tree that performs well? Try it out :)

# **Part 3 - Accumulated Local Effects (ALE)**

Implemented some inefficient code to calculate ALE so you can get a bit clearer view
how it's done (Hopefully I understood it correctly?). Completely optional to study the code of course.

In [ ]:
def get_ALE(feature, model, X, n_neighbourhoods = 20):

  ALE_df = X.copy()

  # Define the neighbourhoods, in this case <n_neighbourhoods> evenly spaced
  # slices that span the 1-D feature space for the selected feature
  feature_neighbourhoods = np.linspace(X[feature].min(),
                                          X[feature].max(),
                                          n_neighbourhoods)

  # Split the feature of interest into the slices
  ALE_df['interval'] = pd.cut(X[feature],
                              feature_neighbourhoods,
                              include_lowest=True)

  ale_vector = []
  n_samples_vector = []

  # For clarity, we loop through the slices
  for g in ALE_df.groupby('interval'):

    feature_interval = g[0]
    X_in_interval = g[1]

    # If we don't have any data instances in the interval,
    # we return nan and skip the calculation
    if (n_samples_in_nei := X_in_interval.shape[0]) == 0:
      ale_vector.append(np.nan)
      continue

    # We calculate the feature value at the left and
    # at the right of the neighbourhood
    z_left = feature_interval.left
    z_right = feature_interval.right

    # We extract the X_train data for the slice (g) at hand
    # and replace all values of <feature> with the rightmost
    # edge of the neighbourhood. All the other features of the
    # data instances are kept unchanged
    X_right = X_in_interval.copy()[X_train.columns]
    X_right[feature] = z_right

    # We do the same for the left edge of the neighbourhood
    X_left = X_in_interval.copy()[X_train.columns]
    X_left[feature] = z_left

    # We get the model predictions for each of the modified data instances
    # and get their difference.
    # We use the model we were passed to do the predictions. It of course
    # assumes the model has implemented a .predict()-method
    y_pred_right = model.predict(X_right)
    y_pred_left = model.predict(X_left)
    delta_y = (y_pred_right - y_pred_left)

    # The local effect for this neighbourhood is then the sum of the
    # differences in predictions divided by the number of samples
    # (i.e. the average difference of predictions in this interval)
    ale_vector.append(round((delta_y.sum() / n_samples_in_nei), 3))
    n_samples_vector.append(n_samples_in_nei)
  # return the neighbourhood vector and the accumulated local effects
  return feature_neighbourhoods[:-1], ale_vector, n_samples_vector

#### Train a quick XGBoost regressor that we can study, hyperparameter tuning is not important for this purpose

In [ ]:
import xgboost

xgb = xgboost.XGBRegressor(n_estimators=300,
                           max_depth=2,
                           random_state=1337)
xgb.fit(X_train, y_train);

print(r2_score(xgb.predict(X_test), y_test).round(3))

### TODO❗ Use the function above or any other method you choose to calculate the ALE of the model xgb, for the feature "total_time". Plot the ALE.

In [ ]:
feature_neighbourhoods, ale_vector, n_samples_vector = ...

### TODO❗ Explain what the ALE plot shows

Answer:

### OPTIONAL TODO❓ Plot a line plot of the n_samples_vector (or like a bar plot to mimic a histogram of the samples)
This histogram would perform the same function as the rug plot in the ALE example plot in the PPT (and Molnar's book). What does it tell us? Does it help us interpret the ALE plot?

### OPTIONAL TODO❓ What would happen if we calculated the ALE for the LinearRegressor we trained in Part 1?

**We encourage you to first think about it and then plot it!**

Answer: